## font transfer (tensorflow version 1 사용) -> tensorflow 2로 변환 작업
- 완료 X


In [ ]:
pip uninstall numpy

Uninstalling numpy-1.19.5:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.19.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.19.5


In [ ]:
pip install numpy==1.19

     |████████████████████████████████| 14.6MB 5.4MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
from __future__ import print_function, division
from __future__ import absolute_import

import argparse
import sys
import numpy as np
import cv2
import os
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import json
import collections
import matplotlib.pyplot as plt

from tensorflow import keras


from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
keras.__version__

'2.2.4-tf'

In [ ]:
np.__version__

'1.19.5'

In [ ]:
# 데이터셋 이미지 처리 및 제작
def crop_image(img):
    image = np.array(img)
    blur = cv2.GaussianBlur(image, ksize=(3,3), sigmaX=0)
    ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
    edged = cv2.Canny(blur, 10, 250)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total = 0
    contours_xy = np.array(contours)
    x_min, x_max = 0,0
    value = list()
    for i in range(len(contours_xy)):
      for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][0])
        x_min = min(value)
        x_max = max(value)
 
    y_min, y_max = 0,0
    value = list()
    for i in range(len(contours_xy)):
      for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][1])
        y_min = min(value)
        y_max = max(value)

    x = x_min
    y = y_min
    w = x_max-x_min
    h = y_max-y_min

    return x, y, w, h

def process_image(img, x, y, w, h, canvas_size):
    new_width = 50
    new_height = int(50 * h / w)
    if new_height > 60:
        new_height = 55
        new_width = int(55 * w / h)
    img = img.crop((x-1, y-1, x+w+1, y+h+1)).resize((new_width,new_height))
    new_left = int((64 - img.width) / 2)
    new_top = int((64 - img.height) / 2)
    result = Image.new("L", (canvas_size, canvas_size), color=255)
    result.paste(img, (new_left, new_top))

    return result

def draw_single_char(ch, font, canvas_size, x_offset, y_offset):
    img = Image.new("L", (150, 150), color=255)
    draw = ImageDraw.Draw(img)
    draw.text((x_offset, y_offset), ch, fill=(0), font=font)
    x, y, w, h = crop_image(img)
    img = process_image(img, x, y, w, h, canvas_size)
    return img


def font2img(ttf, charset, char_size, canvas_size, x_offset, y_offset, folder):
    global X_train, y_train
    font = ImageFont.truetype(ttf, size=char_size)
    count = 0
    for c in charset:
        e = draw_single_char(c, font, canvas_size, x_offset, y_offset)
        if e:
            e.save(folder+c+'.png')
            # X_train.append(np.array(e))
            # y_train.append(count)
            count += 1

In [ ]:
path = './gdrive/MyDrive/hangul/ttf/'
fonts = os.listdir(path)
charset = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
char_size, canvas_size = 100, 64
x_offset, y_offset = 10, 10
label = 1
X_train = []
y_train = []

srcfont = fonts[0]
font2img(path + srcfont, charset, char_size, canvas_size, x_offset, y_offset, 'src/')

dstfont = fonts[1]
font2img(path + dstfont, charset, char_size, canvas_size, x_offset, y_offset, 'dst/')

# for font in fonts:
#     font2img(path + font, charset, char_size, canvas_size, x_offset, y_offset)
#     print('font#%02d 데이터셋 생성 완료' % (label))
#     label += 1

In [ ]:
np.array(Image.open('src/A.png')).shape

(64, 64)

In [ ]:
import matplotlib as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 이미지의 픽셀을 [0, 1]로 normalize하고 src-dst 이미지 연결
def data_preprocess(path='', source_font='src', target_font='dst'):
    source_path = source_font
    source_fonts = list()
    target_path = target_font
    target_fonts = list()
    files = os.listdir(source_path)
    mid1 = np.zeros((64, 64, 1))
    mid2 = np.zeros((64, 64, 1))   
    
    for file in files:
        # src 이미지 로드
        img1 = cv2.imread(source_path+'/'+file)
        np1 = np.array(img1)
        mid1 = np1[:, :, 0]
        mid1 = mid1[:,:,np.newaxis]
        source_fonts.append(mid1)

        # dst 이미지 로드
        img2 = cv2.imread(source_path+'/'+file)
        np2 = np.array(img2)
        mid2 = np2[:, :, 0]
        mid2 = mid2[:,:,np.newaxis]
        target_fonts.append(mid2)

    source_fonts = np.array(source_fonts)
    source_fonts = source_fonts.astype(np.float32)
    source_fonts /= 255

    target_fonts = np.array(target_fonts)
    target_fonts = target_fonts.astype(np.float32)
    target_fonts /= 255

    return source_fonts, target_fonts


class Dataset:
    def __init__(self, source_fonts, target_fonts, test_frac=0.4, val_frac=0.1, scale_func=None):
        # train, validation, test dataset 제작
        self.data_num = int(len(source_fonts))
        self.val_num = int(self.data_num*(1-test_frac)*val_frac)
        self.train_num = int(self.data_num * (1 - test_frac)) - self.val_num
        self.test_num = self.data_num-self.val_num-self.train_num

        self.train = {}
        self.test = {}
        self.valid = {}

        self.train['source_font'] =source_fonts[:self.train_num]
        self.valid['source_font'] =source_fonts[self.train_num: self.train_num+self.val_num]
        self.test['source_font'] = source_fonts[self.train_num + self.val_num:]

        self.train['target_font'] =target_fonts[:self.train_num]
        self.valid['target_font'] =target_fonts[self.train_num: self.train_num+self.val_num]
        self.test['target_font'] =target_fonts[self.train_num + self.val_num:]

    def shuffle_data( self ):
        idx = np.arange(self.train_num)
        np.random.shuffle(idx)
        self.train['source_font'], self.train['target_font'] = self.train['source_font'][idx], self.train['target_font'][idx]

    def get_batches(self, batch_size):
        batch_num = self.train_num // batch_size

        for ii in range(0, batch_num * batch_size, batch_size):
            source_font = self.train['source_font'][ii:ii+batch_size]
            target_font = self.train['target_font'][ii:ii+batch_size]

            yield source_font, target_font

In [ ]:
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

image_dim = (64, 64, 1)
learning_rate = 0.005
batch_size = 32
element_num = batch_size*image_dim[0]*image_dim[1]*image_dim[2]
epochs = 30
beta1 = 0.9
lambda_p = 1
lambda_1 = 10
lambda_2 = 1
lambda_3 = 1
lambda_4 = 30
print_every = 1

LAMDA = 100


def generator(source_font, reuse=False, training=True):
    model = tf.keras.Sequential()
    model = tf.reshape(source_font, (-1, 64, 64, 1)) # reshape 안되면 train.ipyb의 discriminator 함수 참고
    
    # conv1: 64*64*1 -> 64*64*64
    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    conv1 = model

    # conv2: 128*128*64 -> 64*64*128
    model.add(layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(filters=128, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    conv2 = model

    # conv3: 64*64*128 -> 32*32*256
    model.add(layers.Conv2D(filters=128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(filters=256, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    conv3 = model

    # conv4: 32*32*256 -> 16*16*512
    model.add(layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(filters=512, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    conv4 = model

    # conv5: 16*16*512 -> 8*8*512
    model.add(layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # up6: 8*8*512 -> 16*16*512
    model.add(layers.Conv2DTranspose(filters=512, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    up6 = model

    # 16*16*512 + 16*16*512 -> 16*16*1024
    model = tf.concat([conv4, up6], axis=3)

    # conv6: 16*16*1024 -> 16*16*256
    model.add(layers.Conv2D(filters=512, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(filters=256, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # up7: 16*16*256 -> 32*32*256
    model.add(layers.Conv2DTranspose(filters=256, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    up7 = model

    # 32*32*256 + 32*32*256 -> 32*32*512
    model = tf.concat([conv3, up7], axis=3)

    # conv7: 32*32*512 -> 32*32*128
    model.add(layers.Conv2D(filters=256, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(filters=128, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # up8: 32*32*128 -> 64*64*128
    model.add(layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    up8 = model

    # 64*64*128 + 64*64*128 -> 64*64*256
    model = tf.concat([conv2, up8], axis=3)

    # conv8: 64*64*256 -> 64*64*64
    model.add(layers.Conv2D(filters=128, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # up9: 64*64*64 -> 128*128*64
    model.add(layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    up9 = model

    # 128*128*64 + 128*128*64 -> 128*128*128
    model = tf.concat([conv1, up9], axis=3)

    # conv9: 128*128*128 -> 128*128*1
    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(filters=1, kernel_size=3, padding='same'))
    model.add(layers.BatchNormalization())
    model = tf.math.sigmoid(model)

    return model


def discriminator(x, reuse=False):
    # w_init = tf.contrib.layers.xavier_initializer()
    model = tf.keras.Sequential()

    # conv1
    model.add(layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # conv2
    model.add(layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # conv3
    model.add(layers.Conv2D(filters=1024, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # conv4
    model.add(layers.Conv2D(filters=2048, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model = tf.reshape(model, (-1, 8*8*2048))
    model.add(layers.Dense(1))
    model = tf.math.sigmoid(model)

    return model


class TransNN:
    def __init__(self):
        tf.reset_default_graph()

    # 학습을 위한 모델 세팅
    def model_setup(self):
        self.target_fonts = tf.placeholder(
            tf.float32, (None,  64, 64, 1), name='target_fonts')
        self.source_fonts = tf.placeholder(
            tf.float32, (None,  64, 64, 1), name='source_fonts')
        self.target_fonts = tf.Variable()

        self.generated_fonts = generator(self.source_fonts)
        self.real_score = discriminator(self.target_fonts)
        self.fake_score = discriminator(self.generated_fonts, reuse=True)

    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    def model_loss(self):
        # generator 학습을 위한 loss
        gan_loss = loss_object(tf.ones_like(self.fake_score), self.fake_score)
        l1_norm = tf.reduce_mean(tf.abs(self.target_fonts - self.generated_fonts))
        v1_loss = gan_loss + lambda_4 * l1_norm
        v1_loss /= element_num

        # discriminator의 성능을 의한 loss
        real_loss = loss_object(tf.ones_like(self.real_score), self.real_score)
        generated_loss = loss_object(tf.zeros_like(self.fake_score), self.fake_score)
        v2_loss = real_loss + generated_loss

        self.generator_loss =  v1_loss  / batch_size 
        self.discriminator_loss = ( lambda_3 * v2_loss ) / batch_size

        self.d_train_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
        self.g_train_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    def train(self):
        self.model_setup()
        self.model_loss()

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True

        with tf.Session(config=config) as sess:
            sess.run(init)

            for epoch_no in range(epochs):
                dataset.shuffle_data()

                batch_no = 0
                for source_font, target_font in dataset.get_batches(batch_size):
                    batch_no += 1
                    print("Batch No: {}/{} ".format(batch_no, dataset.train_num // batch_size))

                    # generator가 src_img로부터 이미지를 생성
                    sess.run(self.generated_fonts, feed_dict={self.source_fonts: source_font, self.target_fonts: target_font})

                    # discriminator 최적화(optimization)
                    sess.run(self.d_train_opt, feed_dict={self.target_fonts: target_font, self.source_fonts: source_font})

                    # generator 최적화 (optimization)
                    sess.run(self.g_train_opt, feed_dict={self.target_fonts: target_font, self.source_fonts: source_font})

                    if epoch_no % print_every == 0:
                        train_loss_d = self.discriminator_loss.eval(
                            {self.target_fonts: target_font, self.source_fonts: source_font})
                        train_loss_g = self.generator_loss.eval(
                            {self.source_fonts: source_font, self.target_fonts: target_font})

                        print("Epoch {}/{}...".format(epoch_no+1, epochs), "Discriminator Loss: {:.8f}...".format(train_loss_d), "Generator Loss: {:.8f}".format(train_loss_g))

            saver.save(sess, './checkpoints/generator')

    # 학습한 generator 모델 불러서 이미지 생성
    def test(self):
        # self.model_setup()

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True

        with tf.Session(config=config) as sess:
            sess.run(init)

            saver = tf.train.import_meta_graph('./checkpoints/generator.meta')
            saver.restore(sess, tf.train.latest_checkpoint('./checkpoints/'))
            graph = tf.get_default_graph()
            image_no = 0

            for num in range(0, dataset.test_num, batch_size):
                generated_fonts = sess.run(self.generated_fonts, feed_dict={
                                           self.source_fonts: dataset.test['source_font'][num:num+batch_size], self.target_fonts: dataset.test['target_font'][num:num+batch_size]})
                source_fonts = dataset.test['target_font'][num:num+batch_size]

                for batch_no in range(min(batch_size, generated_fonts.shape[0])):
                    generated_font = generated_fonts[batch_no]
                    target_font = source_fonts[batch_no]

                    mid = np.append(generated_font, generated_font, axis=2)
                    mid = np.append(generated_font, mid, axis=2)
                    mid = mid*255
                    mid = mid.astype('uint8')
                    img = Image.fromarray(mid)
                    img.save(fp='./generated_fonts/generated_' +
                             str(image_no) + '.jpg')

                    mid[:, :, :] = target_font
                    mid = mid*255
                    mid = mid.astype('uint8')
                    img = Image.fromarray(mid)
                    img.save(fp='./generated_fonts/target_' +
                             str(image_no)+'.jpg')

                    image_no += 1


# writer=tf.summary.FileWriter(r"./gdrive/MyDrive/font_transfer/models",tf.get_default_graph())
# writer.close()
source_fonts, target_fonts = data_preprocess()
dataset = Dataset(source_fonts, target_fonts, test_frac=0.1, val_frac=0)

net = TransNN()
net.train()

net.test()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.


NotImplementedError: ignored